### *This file allows to initialize a STG spiking neuron as well as neuromodulating them in a robust way with a controller*

# **Useful packages and functions**

In [ ]:
using DifferentialEquations, Plots, Plots.PlotMeasures, LaTeXStrings, Random, Dierckx, DelimitedFiles
include("STG_kinetics.jl") # Loading of STG kinetics of gating variables
include("STG_models.jl") # Loading of STG model
include("STG_utils.jl") # Loading of some utils functions
include("STG_gs_derivatives.jl") # Loading of X_inf derivatives
include("STG_DIC.jl") # Loading of the DIC and compensation algorithm
include("STG_neuromodulation.jl") # Loading of the neuromodulation cells functions
include("STG_large_simulations.jl"); # Loading of the neuromodulation cells functions

# **Global variables**

In [ ]:
# Definition of simulation time (in ms)
const Tfinal = 300000
const tspan  = (0.0, Tfinal)

# Definition of reversal potential values (in mV) and membrane capacitance
const VNa   = 50. # Sodium reversal potential
const VK    = -80. # Potassium reversal potential
const VCa   = 80. # Calcium reversal potential
const VH    = -20. # Reversal potential for the H-current (permeable to both sodium and potassium ions)
const Vleak = -50. # Reversal potential of leak channels
const C     = 1. # Membrane capacitance

# Definition of voltage range for the DICs
const Vmin = -60 
const Vmax = 0
const V    = range(Vmin, stop=Vmax, step=0.01)

# Definition of the number of cells in the random set
const ncells = 200

# Modifying backend GR attributes
gr(guidefontsize=18, legendfontsize=12, margin=5Plots.mm, grid=false)
myApple = RGBA(187/255, 206/255, 131/255, 1)
mySalmon = RGBA(243/255, 124/255, 130/255)
myYellow = RGBA(228/255, 205/255, 121/255, 1)
myBlue = RGBA(131/255, 174/255, 218/255, 1)
myDarkBlue = RGBA(114/255, 119/255, 217/255, 1)
myOrange = RGBA(241/255, 175/255, 113/255, 1)
myPink = RGBA(243/255, 124/255, 130/255, 1)
myPurple = RGBA(169/255, 90/255, 179/255, 1)
myGreen = RGBA(132/255, 195/255, 168/255, 1)
myRed = RGBA(158/255, 3/255, 8/255, 1)
myGray = RGBA(150/255, 150/255, 150/255, 1)
myLightBlue = RGBA(127/255, 154/255, 209/255, 1)
default(fmt = :png)

# Moving average function
moving_average(vs, n, padding) = [sum(vs[i:(i+n-1)])/n for i in 1:padding:(length(vs)-(n-1))];

# **Neuromodulation gCaS and gA overall**

## **Computation of gCaS and gA**

In [ ]:
# Fixing random seed
Random.seed!(544)

# Initial firing pattern
guth = 4.
Vth = -50.
(g_all_init, ICs_th_init) = degeneracy_fixDICs_neuromod(ncells, 5., guth, Vth);
# create a spiking set with max variability in gCaS and gA

In [ ]:
# Definition of parameters
α = 5e-3 # Rate of transfer between intracellular and membrane
β = 5e-3 # Rate of degradation of intracellular proteins
Kp = 3e-4 # Proprtional gain
Ki = 5e-6 # Integral gain
Kt = β / Ki # Anti-windup gain
gsth_sim(t) = -8. # Reference
guth_sim(t) = 4.
u_maxCaS = 1e7 # Maximum value of actuator
u_maxA = 1e7

# Definition of homeostatic parameters
tau_g = 100 # Conductance time constant
tau_Na = 600 # Sodium integral action time constants
Ca_tgt(t) = 125. # Calcium target

# Input current definition
Iapp = 0.

tt = 0. : 0.2 : Tfinal/3;

In [5]:
# g_all_init_blockade = simulate_STG_population_init_blockade(g_all_init, Iapp, tau_Na, tau_g, Ca_tgt,
#                                                             C, α, β, Kp, Ki, Kt, gsth_sim, guth_sim,
#                                                             ICs_th_init, u_maxCaS, u_maxA, tt)

In [6]:
# writedlm("./data/g_all_init_blockade.dat", g_all_init_blockade)

In [5]:
g_all_init_blockade = readdlm("./data/g_all_init_blockade.dat");

In [6]:
gsth_sim2(t) = -8.
guth_sim2(t) = -4.
gNa_matrix_H_blockade, gCaT_matrix_H_blockade, gCaS_matrix_H_blockade, 
gA_matrix_H_blockade, gKCa_matrix_H_blockade, gKd_matrix_H_blockade, 
gleak_matrix_H_blockade, Ca_ma_matrix_H_blockade,
burstiness_before_H_blockade, burstiness_after_H_blockade = simulate_STG_population_H_blockade(g_all_init_blockade, 
                                                   Iapp, tau_Na, tau_g, Ca_tgt, C, α, β, Kp, Ki, Kt, 
                                                   gsth_sim2, guth_sim2, ICs_th_init, u_maxCaS, u_maxA, tt)

Computing... 100%|███████████████████████████████████████| Time: 2:21:42


([3426.770975554893 3462.4306370825784 … 2574.438084509549 2594.8709998765817; 2942.84725397052 3020.367574517673 … 2651.4940911818844 2639.7824822195917; … ; 3858.383957940413 3846.690067684352 … 2448.4703180382007 2467.0300820562925; 3491.5496237483203 3516.556768979467 … 2707.2087450720396 2677.0822581576986], [2.525769072073133 2.552052728275783 … 1.897540319502127 1.9126008024044483; 3.9074900940707193 4.0104209832695386 … 3.5206335910881554 3.505083006209672; … ; 2.464842382644915 2.4573720021343353 … 1.5641505558633129 1.5760070464206128; 1.5764223757371358 1.5877130138615807 … 1.2222952274514083 1.2086932245610136], [16.439762626931323 27.966295903815425 … 30.500697740876024 30.4984683089898; 14.93689787457571 23.176189426341853 … 28.680037610547142 28.681050233928417; … ; 18.158198269793086 31.45434359378551 … 29.322567693501824 29.320372996314433; 19.17086319659958 29.17610577836777 … 30.843003009574563 30.845365349806094], [61.36894664344717 242.63434481885508 … 331.86125671

In [7]:
writedlm("./data/gNa_matrix_H_blockade.dat", gNa_matrix_H_blockade)
writedlm("./data/gCaT_matrix_H_blockade.dat", gCaT_matrix_H_blockade)
writedlm("./data/gCaS_matrix_H_blockade.dat", gCaS_matrix_H_blockade)
writedlm("./data/gA_matrix_H_blockade.dat", gA_matrix_H_blockade)
writedlm("./data/gKCa_matrix_H_blockade.dat", gKCa_matrix_H_blockade)
writedlm("./data/gKd_matrix_H_blockade.dat", gKd_matrix_H_blockade)
writedlm("./data/gleak_matrix_H_blockade.dat", gleak_matrix_H_blockade)
writedlm("./data/Ca_ma_matrix_H_blockade.dat", Ca_ma_matrix_H_blockade)
writedlm("./data/burstiness_before_H_blockade.dat", burstiness_before_H_blockade)
writedlm("./data/burstiness_after_H_blockade.dat", burstiness_after_H_blockade)

In [10]:
# gNa_matrix_H_blockade = readdlm("./data/gNa_matrix_H_blockade.dat")
# gCaT_matrix_H_blockade = readdlm("./data/gCaT_matrix_H_blockade.dat")
# gCaS_matrix_H_blockade = readdlm("./data/gCaS_matrix_H_blockade.dat")
# gA_matrix_H_blockade = readdlm("./data/gA_matrix_H_blockade.dat")
# gKCa_matrix_H_blockade = readdlm("./data/gKCa_matrix_H_blockade.dat")
# gKd_matrix_H_blockade = readdlm("./data/gKd_matrix_H_blockade.dat")
# gleak_matrix_H_blockade = readdlm("./data/gleak_matrix_H_blockade.dat")
# Ca_ma_matrix_H_blockade = readdlm("./data/Ca_ma_matrix_H_blockade.dat");

In [8]:
gsth_sim3(t) = -10.
guth_sim3(t) = 3.
gNa_matrix_CaT_blockade, gCaS_matrix_CaT_blockade, gA_matrix_CaT_blockade, 
gKCa_matrix_CaT_blockade, gKd_matrix_CaT_blockade, gH_matrix_CaT_blockade, 
gleak_matrix_CaT_blockade, Ca_ma_matrix_CaT_blockade,
burstiness_before_CaT_blockade, burstiness_after_CaT_blockade = simulate_STG_population_CaT_blockade(g_all_init_blockade, 
                                                       Iapp, tau_Na, tau_g, Ca_tgt, C, α, β, Kp, Ki, Kt, 
                                                       gsth_sim3, guth_sim3, ICs_th_init, u_maxCaS, u_maxA, tt)

Computing... 100%|███████████████████████████████████████| Time: 3:14:25


([3426.770975554893 3409.7601023911134 … 3312.5828683380255 3310.2643337073755; 2942.84725397052 2982.2495238522915 … 3477.010191106018 3482.9072053842456; … ; 3858.383957940413 3828.935926713052 … 3819.202524552306 3799.462715201388; 3491.5496237483203 3500.229028269236 … 3762.987641628885 3787.7422375931205], [16.43976224470342 21.703440136339466 … 26.75459728704219 26.754534980338455; 14.936897666769278 19.7114257630158 … 29.546550732384212 29.55473140902269; … ; 18.158197809935874 23.998891489465695 … 30.289101562938384 30.370834494840974; 19.170862885869866 24.055583504755155 … 32.059757782895055 31.993176276976165], [61.36893848401657 111.51971491320542 … 162.2889010975145 162.288523155496; 91.94711548007766 131.5191139575198 … 207.2199441651372 207.27731801467954; … ; 56.477205105726426 113.57118680428134 … 175.70271203961042 176.17683299590018; 97.52748108116384 147.00088303699343 … 219.51079661441506 219.05491795484807], [94.67124775325038 94.20128912477075 … 91.51657804642049

In [9]:
writedlm("./data/gNa_matrix_CaT_blockade.dat", gNa_matrix_CaT_blockade)
writedlm("./data/gCaS_matrix_CaT_blockade.dat", gCaS_matrix_CaT_blockade)
writedlm("./data/gA_matrix_CaT_blockade.dat", gA_matrix_CaT_blockade)
writedlm("./data/gKCa_matrix_CaT_blockade.dat", gKCa_matrix_CaT_blockade)
writedlm("./data/gKd_matrix_CaT_blockade.dat", gKd_matrix_CaT_blockade)
writedlm("./data/gH_matrix_CaT_blockade.dat", gH_matrix_CaT_blockade)
writedlm("./data/gleak_matrix_CaT_blockade.dat", gleak_matrix_CaT_blockade)
writedlm("./data/Ca_ma_matrix_CaT_blockade.dat", Ca_ma_matrix_CaT_blockade)
writedlm("./data/burstiness_before_CaT_blockade.dat", burstiness_before_CaT_blockade)
writedlm("./data/burstiness_after_CaT_blockade.dat", burstiness_after_CaT_blockade)

In [13]:
# gNa_matrix_CaT_blockade = readdlm("./data/gNa_matrix_CaT_blockade.dat")
# gCaS_matrix_CaT_blockade = readdlm("./data/gCaS_matrix_CaT_blockade.dat")
# gA_matrix_CaT_blockade = readdlm("./data/gA_matrix_CaT_blockade.dat")
# gKCa_matrix_CaT_blockade = readdlm("./data/gKCa_matrix_CaT_blockade.dat")
# gKd_matrix_CaT_blockade = readdlm("./data/gKd_matrix_CaT_blockade.dat")
# gH_matrix_CaT_blockade = readdlm("./data/gH_matrix_CaT_blockade.dat")
# gleak_matrix_CaT_blockade = readdlm("./data/gleak_matrix_CaT_blockade.dat")
# Ca_ma_matrix_CaT_blockade = readdlm("./data/Ca_ma_matrix_CaT_blockade.dat");

In [10]:
gCaT_matrix_Na_blockade, gCaS_matrix_Na_blockade, gA_matrix_Na_blockade, 
gKCa_matrix_Na_blockade, gKd_matrix_Na_blockade, gH_matrix_Na_blockade, 
gleak_matrix_Na_blockade, Ca_ma_matrix_Na_blockade,
burstiness_before_Na_blockade, burstiness_after_Na_blockade = simulate_STG_population_Na_blockade(g_all_init_blockade, 
                                                     Iapp, tau_Na, tau_g, Ca_tgt, C, α, β, Kp, Ki, Kt, 
                                                     gsth_sim, guth_sim, ICs_th_init, u_maxCaS, u_maxA, tt)

Computing... 100%|███████████████████████████████████████| Time: 0:37:05


([2.5257690720731336 2.609808402929915 … 10.514813238201238 10.567304180948181; 3.9074900940707193 4.059326538219877 … 15.047584019144605 15.098349001135622; … ; 2.4648423826449153 2.5334339171810774 … 9.396493441208936 9.44020910434075; 1.5764223757371358 1.6340254180136684 … 8.353167149466223 8.421212878290278], [16.439761865923305 15.836791639475486 … 63.10065451793261 63.42575226813802; 14.936897327683248 14.433963774741322 … 53.34356093177716 53.53438745387347; … ; 18.158197386205604 17.454233715538102 … 63.358376436203514 63.66949464332414; 19.170862528241646 18.574687829391486 … 94.09468314487862 94.87501613537609], [61.368935573615424 66.16026347623874 … 304.83533934669185 306.46949746443613; 91.94711320971177 96.22951239120187 … 392.55514442183465 393.9876402559009; … ; 56.47720177733336 61.75927344242907 … 261.22386298022724 262.5423346154373; 97.5274781995042 102.55996529966772 … 574.5350225390706 579.3831276248758], [94.67124775325038 97.82122230972531 … 394.11777591206936 

In [11]:
writedlm("./data/gCaT_matrix_Na_blockade.dat", gCaT_matrix_Na_blockade)
writedlm("./data/gCaS_matrix_Na_blockade.dat", gCaS_matrix_Na_blockade)
writedlm("./data/gA_matrix_Na_blockade.dat", gA_matrix_Na_blockade)
writedlm("./data/gKCa_matrix_Na_blockade.dat", gKCa_matrix_Na_blockade)
writedlm("./data/gKd_matrix_Na_blockade.dat", gKd_matrix_Na_blockade)
writedlm("./data/gH_matrix_Na_blockade.dat", gH_matrix_Na_blockade)
writedlm("./data/gleak_matrix_Na_blockade.dat", gleak_matrix_Na_blockade)
writedlm("./data/Ca_ma_matrix_Na_blockade.dat", Ca_ma_matrix_Na_blockade)
writedlm("./data/burstiness_before_Na_blockade.dat", burstiness_before_Na_blockade)
writedlm("./data/burstiness_after_Na_blockade.dat", burstiness_after_Na_blockade)

In [16]:
# gCaT_matrix_Na_blockade = readdlm("./data/gCaT_matrix_Na_blockade.dat")
# gCaS_matrix_Na_blockade = readdlm("./data/gCaS_matrix_Na_blockade.dat")
# gA_matrix_Na_blockade = readdlm("./data/gA_matrix_Na_blockade.dat")
# gKCa_matrix_Na_blockade = readdlm("./data/gKCa_matrix_Na_blockade.dat")
# gKd_matrix_Na_blockade = readdlm("./data/gKd_matrix_Na_blockade.dat")
# gH_matrix_Na_blockade = readdlm("./data/gH_matrix_Na_blockade.dat")
# gleak_matrix_Na_blockade = readdlm("./data/gleak_matrix_Na_blockade.dat")
# Ca_ma_matrix_Na_blockade = readdlm("./data/Ca_ma_matrix_Na_blockade.dat");

In [12]:
gsth_sim2(t) = -8.
guth_sim2(t) = -4.
gNa_matrix_KCa_blockade, gCaT_matrix_KCa_blockade, gCaS_matrix_KCa_blockade, 
gA_matrix_KCa_blockade, gKd_matrix_KCa_blockade, gH_matrix_KCa_blockade, 
gleak_matrix_KCa_blockade, Ca_ma_matrix_KCa_blockade,
burstiness_before_KCa_blockade, burstiness_after_KCa_blockade = simulate_STG_population_KCa_blockade(g_all_init_blockade, 
                                                   Iapp, tau_Na, tau_g, Ca_tgt, C, α, β, Kp, Ki, Kt, 
                                                   gsth_sim2, guth_sim2, ICs_th_init, u_maxCaS, u_maxA, tt)

Computing... 100%|███████████████████████████████████████| Time: 2:33:14


([3426.770975554893 3320.2103936284902 … 2894.3845823249344 2900.7941223810435; 2942.8472539705203 2823.5789401902985 … 1777.9386429408535 1795.2329619437596; … ; 3858.383957940413 3723.5591547016866 … 2916.2041527921974 2916.197438305447; 3491.5496237483203 3405.0951228191257 … 3060.536672553817 3060.2206238146846], [2.525769072073133 2.4472264953874316 … 2.133363190266191 2.1380874680645596; 3.907490094070719 3.7491265385025776 … 2.360733342780314 2.3836966073862156; … ; 2.4648423826449153 2.378712569521851 … 1.8629518654967818 1.862947576096921; 1.5764223757371358 1.537388472618309 … 1.3818215440966832 1.3816788492024765], [16.43976380856882 45.74944239298899 … 67.22833536358135 67.39529350884044; 14.936898914652105 38.75242518290687 … 40.21804037571425 40.24756051347859; … ; 18.15819959040469 51.355552509438866 … 68.08287905457799 68.08247750333142; 19.170864426723202 47.78211872187722 … 71.02594766287179 71.0257505606484], [61.368965295591885 525.9644172556609 … 894.2240108000433 

In [13]:
writedlm("./data/gNa_matrix_KCa_blockade.dat", gNa_matrix_KCa_blockade)
writedlm("./data/gCaT_matrix_KCa_blockade.dat", gCaT_matrix_KCa_blockade)
writedlm("./data/gCaS_matrix_KCa_blockade.dat", gCaS_matrix_KCa_blockade)
writedlm("./data/gA_matrix_KCa_blockade.dat", gA_matrix_KCa_blockade)
writedlm("./data/gKd_matrix_KCa_blockade.dat", gKd_matrix_KCa_blockade)
writedlm("./data/gH_matrix_KCa_blockade.dat", gH_matrix_KCa_blockade)
writedlm("./data/gleak_matrix_KCa_blockade.dat", gleak_matrix_KCa_blockade)
writedlm("./data/Ca_ma_matrix_KCa_blockade.dat", Ca_ma_matrix_KCa_blockade)
writedlm("./data/burstiness_before_KCa_blockade.dat", burstiness_before_KCa_blockade)
writedlm("./data/burstiness_after_KCa_blockade.dat", burstiness_after_KCa_blockade)

In [19]:
# gNa_matrix_KCa_blockade = readdlm("./data/gNa_matrix_KCa_blockade.dat")
# gCaT_matrix_KCa_blockade = readdlm("./data/gCaT_matrix_KCa_blockade.dat")
# gCaS_matrix_KCa_blockade = readdlm("./data/gCaS_matrix_KCa_blockade.dat")
# gA_matrix_KCa_blockade = readdlm("./data/gA_matrix_KCa_blockade.dat")
# gKd_matrix_KCa_blockade = readdlm("./data/gKd_matrix_KCa_blockade.dat")
# gH_matrix_KCa_blockade = readdlm("./data/gH_matrix_KCa_blockade.dat")
# gleak_matrix_KCa_blockade = readdlm("./data/gleak_matrix_KCa_blockade.dat")
# Ca_ma_matrix_KCa_blockade = readdlm("./data/Ca_ma_matrix_KCa_blockade.dat");